In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [5]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(y_train.shape)


(60000, 28, 28)
(60000,)


In [6]:
x_train = x_train.reshape(-1, 28*28).astype("float32") / 255.0 # we divided by 255 to normalize the values to be between 0 and 1 rather than 0 and 255
x_test = x_test.reshape(-1, 28*28).astype("float32") / 255.0 
# the conversion from numpy to tensor is done automatically

In [ ]:
# Sequential api